In [73]:
import pandas as pd
pd.set_option("display.max_columns", None)

dtype = {
    "tweetid": str
}

# creater object for iteration with chunksize of 50,000 observations
chunks = pd.read_csv(
    "russian_linked_tweets_csv_hashed.csv", 
    chunksize=50000,
    dtype=dtype,
    low_memory=False)

In [74]:
from tqdm import tqdm

# empty list to store chunks
chunk_list = []

# iterate through chunks, appending to list
for chunk in tqdm(chunks):
    chunk_list.append(chunk)
    
# concat list into dataframe
df = pd.concat(chunk_list)

19it [00:07,  2.38it/s]


In [75]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920761 entries, 0 to 920760
Data columns (total 31 columns):
tweetid                     920761 non-null object
userid                      920761 non-null object
user_display_name           920761 non-null object
user_screen_name            920761 non-null object
user_reported_location      912810 non-null object
user_profile_description    901370 non-null object
user_profile_url            329438 non-null object
follower_count              920761 non-null int64
following_count             920761 non-null int64
account_creation_date       920761 non-null object
account_language            920761 non-null object
tweet_language              918184 non-null object
tweet_text                  920761 non-null object
tweet_time                  920761 non-null object
tweet_client_name           920761 non-null object
in_reply_to_userid          76106 non-null object
in_reply_to_tweetid         72380 non-null float64
quoted_tweet_tweetid     

In [76]:
df.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_userid,in_reply_to_tweetid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,894109669869985793,4125840934,Inside Syria MC,Inside_Syria,Syria,Collecting information about the Syrian confli...,https://t.co/x9PTLhaz76,19816,808,2015-11-06,en,en,New Convoy of #SyrianArmy Arrived in #Hama #In...,2017-08-06 08:15,WordPress.com,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,0.0,0.0,"['SyrianArmy', 'Hama', 'InsideSyriaMC']",['http://en.insidesyriamc.com/2017/08/06/new-c...,[],NaN
1,839851611656761344,4125840934,Inside Syria MC,Inside_Syria,Syria,Collecting information about the Syrian confli...,https://t.co/x9PTLhaz76,19816,808,2015-11-06,en,ar,الجيش السوري يتقدم في محيط حلب #سورية_النظر_من...,2017-03-09 14:53,WordPress.com,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,0.0,0.0,['سورية_النظر_من_الداخل'],['http://insidesyriamc.com/2017/03/09/%d8%a7%d...,[],NaN
2,841662351510249472,4125840934,Inside Syria MC,Inside_Syria,Syria,Collecting information about the Syrian confli...,https://t.co/x9PTLhaz76,19816,808,2015-11-06,en,ar,#الجعفري: نحن حريصون كل الحرص على إنجاح مسار #...,2017-03-14 14:48,WordPress.com,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,0.0,0.0,"['الجعفري', 'أستانا', 'سورية_النظر_من_الداخل']",['http://insidesyriamc.com/2017/03/14/%d8%a7%d...,[],NaN
3,820599201495187456,4125840934,Inside Syria MC,Inside_Syria,Syria,Collecting information about the Syrian confli...,https://t.co/x9PTLhaz76,19816,808,2015-11-06,en,en,#Iraq:\nMore than 30 civilians killed in a U.@...,2017-01-15 11:51,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,1.0,0.0,"['Iraq', 'airstrike']",[],[2876420003],NaN
4,549829105483808768,EjLn9BPgSPWKPhEnebvOIWJ2quALQVMNPVfFgGyzG+8=,EjLn9BPgSPWKPhEnebvOIWJ2quALQVMNPVfFgGyzG+8=,EjLn9BPgSPWKPhEnebvOIWJ2quALQVMNPVfFgGyzG+8=,"New York, USA",experienced,NaN,129,194,2014-05-30,en,en,"Those who truly know me, know that my true pay...",2014-12-30 07:27,Twitter Web Client,NaN,NaN,NaN,False,NaN,NaN,absent,absent,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [77]:
# generate list of columns
list(df.columns)

['tweetid',
 'userid',
 'user_display_name',
 'user_screen_name',
 'user_reported_location',
 'user_profile_description',
 'user_profile_url',
 'follower_count',
 'following_count',
 'account_creation_date',
 'account_language',
 'tweet_language',
 'tweet_text',
 'tweet_time',
 'tweet_client_name',
 'in_reply_to_userid',
 'in_reply_to_tweetid',
 'quoted_tweet_tweetid',
 'is_retweet',
 'retweet_userid',
 'retweet_tweetid',
 'latitude',
 'longitude',
 'quote_count',
 'reply_count',
 'like_count',
 'retweet_count',
 'hashtags',
 'urls',
 'user_mentions',
 'poll_choices']

There is a lot of information here. For the purposes of this project -- and to keep things as uniform as possible between the Verified and Russian data sets -- we'll only use a portion of these columns. Namely, we'll use:
- `tweetid` (represented by `id_str` in Verified account data)
- `user_screen_name` (represented by `screen_name` in Verified account data)
- `tweet_time` (represented by `created_at` in Verified account data)
- `tweet_language` (represented by `lang` in Verified account data)
- `tweet_client_name` (represented by `source` in Verified account data)
- `retweet_count`
- `like_count` (represented by `favorite_count` in Verified account data)
- `tweet_text` (represented by `full_text` in Verified account data)

In [78]:
cols_interest = [
    "tweetid",
    "user_screen_name",
    "tweet_time",
    "tweet_language",
    "tweet_client_name",
    "retweet_count",
    "like_count",
    "tweet_text"
]

# grab only columns we're interested in
df = df[cols_interest].copy()

In [79]:
df.head()

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
0,894109669869985793,Inside_Syria,2017-08-06 08:15,en,WordPress.com,0.0,0.0,New Convoy of #SyrianArmy Arrived in #Hama #In...
1,839851611656761344,Inside_Syria,2017-03-09 14:53,ar,WordPress.com,0.0,0.0,الجيش السوري يتقدم في محيط حلب #سورية_النظر_من...
2,841662351510249472,Inside_Syria,2017-03-14 14:48,ar,WordPress.com,0.0,0.0,#الجعفري: نحن حريصون كل الحرص على إنجاح مسار #...
3,820599201495187456,Inside_Syria,2017-01-15 11:51,en,Twitter Web Client,0.0,1.0,#Iraq:\nMore than 30 civilians killed in a U.@...
4,549829105483808768,EjLn9BPgSPWKPhEnebvOIWJ2quALQVMNPVfFgGyzG+8=,2014-12-30 07:27,en,Twitter Web Client,0.0,0.0,"Those who truly know me, know that my true pay..."


Looks like this data set contains other languages. Lets see the breakdown by each using the `tweet_language` column.

In [80]:
# convert tweet_language column to category type
df["tweet_language"] = df["tweet_language"].astype("category")

In [81]:
# get the value counts for top 10 languages
round(df["tweet_language"].value_counts(normalize=True) * 100, 4)[:10]

en     80.7741
und    16.2598
fr      0.9499
ru      0.7810
ar      0.2129
es      0.1675
in      0.1233
tr      0.0990
tl      0.0737
de      0.0673
Name: tweet_language, dtype: float64

Looks like the vast majority of observations are in English or are undefined (i.e., `und`). Let's see if we might be able to reclassify these undefined observations using [`googletrans`](https://github.com/ssut/py-googletrans).

In [24]:
# install googletrans library (if needed)
#!pip install googletrans

In [67]:
from googletrans import Translator

# instantiate translator object
translator = Translator()

In [65]:
# grab observations with und tweet language
undefined = df[df["tweet_language"] == "und"]
undefined[:3]

,tweetid,user_screen_name,tweet_time,tweet_language,tweet_client_name,retweet_count,like_count,tweet_text
5,548749959605280769,yjxwJBdqQgtGbJIozVTuYw23B+DY557Nz8lZeJ4jsJg=,2014-12-27 07:59,und,Twitter for Android,0.0,0.0,http://t.co/WU6zpuKdjE
11,846188781770190848,SamInfidel,2017-03-27 02:35,und,Twitter for iPhone,1.0,0.0,#SuddenJihadSyndrome #ReligionOfShit #BanTheQu...
19,584085161253281792,CathyTo47590555,2015-04-03 20:08,und,Twitter for iPhone,0.0,0.0,http://t.co/qlJAXUnjTn


In [71]:
#undefined["tweet_text"][:5].map(translator.detect).apply(getattr, args=("lang"))

Due to consistent errors when trying to detect the language, we'll just drop all observations except those in English. Yes, this eliminates quite a bit of data however, since all our Verified data is in English, I think this is the best route to go. Additionally, using only English to start out decreases the complexity by not having to do any translation, and we're not even sure if we'll be able to build a good model in the first place.

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920761 entries, 0 to 920760
Data columns (total 8 columns):
tweetid              920761 non-null object
user_screen_name     920761 non-null object
tweet_time           920761 non-null object
tweet_language       918184 non-null category
tweet_client_name    920761 non-null object
retweet_count        914726 non-null float64
like_count           914726 non-null float64
tweet_text           920761 non-null object
dtypes: category(1), float64(2), object(5)
memory usage: 50.1+ MB


In [84]:
# grab subset of observations that are only in english
english = df[df["tweet_language"] == "en"]

In [85]:
english.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 741655 entries, 0 to 920760
Data columns (total 8 columns):
tweetid              741655 non-null object
user_screen_name     741655 non-null object
tweet_time           741655 non-null object
tweet_language       741655 non-null category
tweet_client_name    741655 non-null object
retweet_count        736139 non-null float64
like_count           736139 non-null float64
tweet_text           741655 non-null object
dtypes: category(1), float64(2), object(5)
memory usage: 46.0+ MB
